In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException
import time
import re

from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager


# use GeckoDriver manager to access firefox page via gecko
executable = FirefoxService(GeckoDriverManager().install())

driver = webdriver.Firefox(service=executable)

[WDM] - Downloading: 19.2kB [00:00, 10.0MB/s]                   


In [3]:
url_base = "https://www.indeed.com/jobs?q="

position_type = ['software engineer', 'full time']
search = '+'.join(' '.join(position_type).split())

url_search = url_base + search

In [11]:
search = 'q=software+engineer&l=united+states&sc=fulltime&fromage=28'

In [17]:
all_titles = []
all_companies = []
all_loc = []
all_salary = []
all_type = []
all_text = []

# page = driver.get(url_search)   
page = driver.get('https://www.indeed.com/jobs?'+search+'&start=10')
# page = driver.get('https://www.indeed.com/jobs?'+search)
driver.implicitly_wait(5)

pg = 0

# pull job cards, excluding spacers and final blank card
to_click = [1,2,3,4,5,7,8,9,10,11,13,14,15,16,17]

In [4]:
while pg < 3:
    
    time.sleep(5)
    # sort by date
    try:
        date_sort = driver.find_element(By.CSS_SELECTOR, ".jobsearch-DesktopSort > span:nth-child(2) > a:nth-child(2)")
        date_sort.click()
        driver.implicitly_wait(10)
        
    except:
        pass


    for card in to_click:
        
        time.sleep(2)
        
        card_search = ".jobsearch-ResultsList > li:nth-child({})".format(card) 
        job_card = driver.find_element(By.CSS_SELECTOR,card_search)
        
        try:
            job_card.click()
        
        except ElementClickInterceptedException:
            x_out = driver.find_element(By.CSS_SELECTOR, ".icl-CloseButton")
            x_out.click()
            job_card.click()
            
        elem = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID,"jobsearch-ViewjobPaneWrapper")) #This is a dummy element
        )
        
        desc = driver.find_element(By.CLASS_NAME, "jobsearch-jobDescriptionText")
        job_text = desc.text
        all_text.append(job_text)

        soup = BeautifulSoup(job_card.get_attribute('innerHTML'), 'html.parser')

        # get html from root node
        soup_children = list(soup.children)
        html = soup_children[0]    

        # Get head from html 
        html_children = list(html.children)
        head = html_children[0]

        # get job card stats
        job_title = head.find('h2').text
        company = head.find("span", {"class":"companyName"}).text
        location = head.find("div",{"class":"companyLocation"}).text

        attr = head.find("div",{"class":"attribute_snippet"}).text
        if sum(map(str.isdigit, attr)) > 2:
            salary = attr
        else:
            salary = 'n/a'

        all_titles.append(job_title)
        all_companies.append(company)
        all_loc.append(location)
        all_salary.append(salary)
        
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
        
    next_page = driver.find_element(By.CSS_SELECTOR, '[data-testid="pagination-page-next"]')
    
    
    try:
        next_page.click()
        
    except ElementClickInterceptedException:
        x_out = driver.find_element(By.CSS_SELECTOR, ".icl-CloseButton")
        x_out.click()
        next_page.click()

    pg += 1
    driver.implicitly_wait(10)
      

In [5]:
data = {
    'title':all_titles,
    'company':all_companies,
    'location':all_loc,
    'salary':all_salary,
    'description':all_text
        }

df = pd.DataFrame(data)

df

,title,company,location,salary,description
0,Software Engineer,"RLA Engineering, LLC",Remote,$63.40 - $65.60 an hour,Job Title: Software Engineer Department: WS\nE...
1,Windows Kernel Developer - 100% Remote - Full ...,Sohum,Remote,"$150,000 - $160,000 a year","Role & Responsibilities:\nDesign, develop, tes..."
2,Software Engineer,Caterpillar,"Chicago, IL 60622 (West Town area)",n/a,Career Area:\nDigital\nJob Description:\nEEO/A...
3,C++ Software Engineer,Asset Inventories Inc.,"Manhattan, NY","$130,000 - $160,000 a year",We are seeking a strong C++ low latency develo...
4,Senior Software Engineer,Caterpillar,"Peoria, IL",n/a,Career Area:\nDigital\nJob Description:\nJOB P...
5,Software Engineer III,Risk Solutions,"Hybrid remote in Berkeley Heights, NJ",n/a,"Job Summary\nXpertHR, a LexisNexis Risk Soluti..."
6,Software Engineer,Ford Motor Company,"Dearborn, MI 48126",n/a,"At Ford Motor Company, we believe freedom of m..."
7,Software Engineer,Quevera LLC,"Annapolis Junction, MD 20701",n/a,Job Description:\n\nQuevera is seeking a Softw...
8,EdTech Software Front-End Engineer,Cambium Learning Group,Remote,n/a,Job Overview:\nThis position works to design/d...
9,Full Stack Senior Software Engineer - .NET (US...,Risk Solutions,Remote in Oklahoma,n/a,Explore how you can grow\nThe future of food s...
